In [1]:
import pathlib
import torch
import torchvision.models as models
import torch.nn as nn
import os

In [2]:
filepath = pathlib.Path.cwd()
edited_components_dir = os.path.join(filepath, "..", "edited_components")

if not os.path.exists(edited_components_dir):
    os.makedirs(edited_components_dir)

# YOLO

For our object detection we are using YOLOv8. We are actually using 2 seperate models for person and face detectinon. To apply the unified backbone into the YOLOv8 model branches, we need to modify the YOLOv8 model to accept the backbone's output as an input. To do that, we need to remove the image input layers from the YOLOv8 model and replace it with the backbone's output. We will work on the backbone's output later, for now, we neeed simply need to remove the image input layers from the YOLOv8 model and add a small adapter layer to replace it.

In [3]:
from ultralytics import YOLO

In [4]:
filepath = pathlib.Path.cwd()

model_path = os.path.join(filepath, "..", "component_models", "yolov8n.pt")
yolo_model = YOLO(model_path)

In [5]:
# print(yolo_model)

Printing out YOLO shows that the first two layers are for augmenting the image to be processed. We can remove these two layers and use the rest of the network to process the image. 

In [ ]:
class CustomYOLO(nn.Module):
    def __init__(self, yolo_model, backbone_channels=2048):
        super(CustomYOLO, self).__init__()

        # Gradual channel reduction with normalization and activation
        self.adapter = nn.Sequential(
            nn.Conv2d(backbone_channels, 512, kernel_size=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 128, kernel_size=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 32, kernel_size=1)
        )
        
        # Remove the first two layers of YOLO, which are for processing input
        self.yolo = nn.Sequential(*list(yolo_model.model.model)[2:])
        
    def forward(self, backbone_features):
        x = self.adapter(backbone_features)
        x = self.yolo(x)
        return x
    
    def forward(self, backbone_features):
        x = self.adapter(backbone_features)
        detections = self.yolo(x)
        
        # if self.training:
        return detections
        
        # TODO: If inference, process detections
        # processed_detections = []
        # for detection in detections:
        #     processed = self._process_detection(detection)
        #     processed_detections.append(processed)
        # return processed_detections

In [7]:
model_path = os.path.join(filepath, "..", "component_models", "yolov8n.pt")
yolo_model = YOLO(model_path)
person_detect_branch = CustomYOLO(yolo_model)
torch.save(person_detect_branch, os.path.join(filepath, "..", "edited_components", "custom_yolo.pth"))

In [8]:
model_path = os.path.join(filepath, "..", "component_models", "yolov8n-face.pt")
yolo_face_model = YOLO(model_path)
face_detect_branch = CustomYOLO(yolo_face_model)
torch.save(face_detect_branch, os.path.join(filepath, "..", "edited_components", "custom_yolo_face.pth"))

# AdaFace

To recognize faces we are using AdaFace. This is a similar process to YOLO, where we need to remove the image input layers from the model and replace it with the backbone's output. To do this, we have copied the AdaFace model PyTorch code and modified it to accept the backbone's output as an input. We apply the same forward as the original code to keep the same functionality.

In [9]:
import sys
import os

current_dir = os.getcwd()

project_root = os.path.abspath(os.path.join(current_dir, '..', 'libs'))

if project_root not in sys.path:
    sys.path.append(project_root)

In [10]:
import net_adaface
from net_adaface import build_model
from head_adaface import build_head

AdaFace has stored the weights of the model as a state dict. Due to this, we need to load the weights of the model into the original model first and then transfer the weights to the modified model.

In [11]:
class CustomAdaFace(nn.Module):
    def __init__(self, pretrained_path, config):
        super(CustomAdaFace, self).__init__()
        
        # Load the complete pre-trained AdaFace model
        self.adaface_model = build_model(model_name=config.arch)
        
        checkpoint = torch.load(pretrained_path)
        if 'state_dict' in checkpoint:
            state_dict = checkpoint['state_dict']
        else:
            state_dict = checkpoint

        state_dict = {k.replace('module.', ''): v for k, v in state_dict.items()}
        
        
        # Load weights except for input layer
        filtered_state_dict = {k: v for k, v in state_dict.items() 
                             if not k.startswith('input_layer')}
        self.adaface_model.load_state_dict(filtered_state_dict, strict=False)
        
        # Replace input layer with an adapter that takes backbone features
        self.adaface_model.input_layer = nn.Sequential(
            nn.Conv2d(2048, 64, kernel_size=1),  # First reduce channels
            nn.BatchNorm2d(64),
            nn.PReLU(64),  # AdaFace uses PReLU
            # Additional layers to match the processing of original input_layer
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.PReLU(64)
        )
        
        # Original AdaFace head
        self.head = build_head(
            head_type=config.head,
            embedding_size=512,
            class_num=config.num_classes,
            m=config.m,
            h=config.h,
            t_alpha=config.t_alpha,
            s=config.s
        )
    
    def forward(self, x, labels=None):
        # Forward through modified AdaFace model
        embeddings, norms = self.adaface_model(x)
        
        if labels is not None:
            return self.head(embeddings, norms, labels)
        return embeddings, norms

In [12]:
class Config:
    def __init__(self):
        self.arch = 'ir_50'
        self.head = 'adaface'
        self.num_classes = 1000  # Update this based on your dataset
        self.m = 0.4
        self.h = 0.333
        self.t_alpha = 0.01
        self.s = 64.0

config = Config()

ada_face_model_path = os.path.join(filepath, "..", "component_models", "adaface_ir50_ms1mv2.ckpt")

In [13]:
face_rec_branch = CustomAdaFace(ada_face_model_path, config)
torch.save(face_rec_branch, os.path.join(filepath, "..", "edited_components", "custom_ada_face.pth"))

/home/william/git_projects/Person-Recognition-for-Pose-Estimation/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



\AdaFace with the following property
self.m 0.4
self.h 0.333
self.s 64.0
self.t_alpha 0.01


# ViT Pose

ViT Pose is a model that detects human poses. The model is avalible on Hugging Face in the transformers library. This is very useful, but we need to take a slightly different approach to use it. Instead of copying over the model to a custom version, we instead create a new model that simply has the adaper layer followed by the ViT Pose model. We then edit the patch embedding layer to an identity matrix to simply skip the layer. This makes it a much simpler process than the previous two models.

In [14]:
from transformers import AutoProcessor, VitPoseForPoseEstimation

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
class CustomVitPose(nn.Module):
    def __init__(self, vit_pose_model: VitPoseForPoseEstimation, backbone_channels=2048):
        super(CustomVitPose, self).__init__()
        
        # Get the hidden size from the model's config
        hidden_size = vit_pose_model.backbone.config.hidden_size
        
        self.adapter = nn.Sequential(
            nn.Conv2d(backbone_channels, hidden_size, kernel_size=1),
            nn.LayerNorm([hidden_size]),
            nn.ReLU(),
            nn.Conv2d(hidden_size, hidden_size, kernel_size=3, padding=1),
            nn.LayerNorm([hidden_size]),
        )
        
        self.vit_pose = vit_pose_model
        self.vit_pose.backbone.embeddings.patch_embeddings = nn.Identity()

    def forward(self, backbone_features):
        x = self.adapter(backbone_features)
        return self.vit_pose(x)


In [16]:
hf_vit_pose_model = VitPoseForPoseEstimation.from_pretrained("usyd-community/vitpose-base-simple", device_map=device)
pose_detect_branch = CustomVitPose(hf_vit_pose_model)
torch.save(pose_detect_branch, os.path.join(filepath, "..", "edited_components", "custom_vit_pose.pth"))

# Backbone

For our backbone we chose Resnet50. We chose Resnet50 because it is a well known and well tested backbone that has been used in many object detection models. This class copies the Resnet50 model format, exept for the last layer. Instead of a classifyer output, we have branches depending on the task. The task is passed into the forward funciton, and only the relevant branch's connection is returned. The full multitask branch is not processed at this stage, but due to the seperate architectures between YOLOv8, AdaFace, and ViT Pose, we need the backbone to have seperate outputs to properly provide valid inputs to each model.

As opposed to downloading the backbone via `download_models.ipynb`, we can instead take the simpler approach of using the `torchvision` library to download the model. This is done by calling `torchvision.models.resnet50(pretrained=True)`. This will download the model and load the pretrained weights. We can then modify the model to have the desired outputs.

In [17]:
resnet_model = models.resnet50(pretrained=True)

/home/william/git_projects/Person-Recognition-for-Pose-Estimation/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/william/git_projects/Person-Recognition-for-Pose-Estimation/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [18]:
# print(resnet_model)

As stated above, we copy the structure of Resnet50, but modify the last layer to have inputs for the individual branches.

In [19]:
class MultiTaskResNetFeatureExtractor(nn.Module):
    def __init__(self, original_model):
        super(MultiTaskResNetFeatureExtractor, self).__init__()
        
        # Just the core ResNet layers
        self.conv1 = original_model.conv1
        self.bn1 = original_model.bn1
        self.relu = original_model.relu
        self.maxpool = original_model.maxpool
        self.layer1 = original_model.layer1
        self.layer2 = original_model.layer2
        self.layer3 = original_model.layer3
        self.layer4 = original_model.layer4

    def forward(self, x):
        # Basic feature extraction
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        return x  # Output shape: [batch_size, 2048, H/32, W/32]

In [20]:
feature_extractor = MultiTaskResNetFeatureExtractor(resnet_model)
torch.save(feature_extractor, os.path.join(filepath, "..", "edited_components", "resnet_feature_extractor.pth"))

# Combined Model

Now that we have all of the branches and the backbone of the model, we can now combine them all into a single multi-task model.

In [ ]:
class CombinedModel(nn.Module):
    def __init__(self, backbone, yolo_face, yolo_person, ada_face, vit_pose):
        super(CombinedModel, self).__init__()

        if any(model_args is None for model_args in [backbone, yolo_face, yolo_person, ada_face, vit_pose]):
            raise ValueError("All models must be provided")

        self.current_task = 'person_detection'
        self.backbone = backbone
        self.yolo_face = yolo_face
        self.yolo_person = yolo_person
        self.ada_face = ada_face
        self.vit_pose = vit_pose

    def set_task(self, task_name):
        supported_tasks = ['face_detection', 'person_detection', 'pose_estimation', 'face_identification']
        if task_name not in supported_tasks:
            raise ValueError(f"Task {task_name} not supported. Available tasks: {', '.join(supported_tasks)}")
        self.current_task = task_name
        
    def forward(self, x):
        # Get features from backbone for the task
        features = self.backbone(x)

        # Use the correct branch for the inputted task
        # torch.where is used here for efficiency
        return torch.where(self.current_task == 'pose_estimation',
            self.vit_pose(features),
            torch.where(self.current_task == 'person_detection',
                self.yolo_person(features),
                torch.where(self.current_task == 'face_detection',
                    self.yolo_face(features),
                    self.ada_face(features)
                )
            )
        )

In [22]:
combined_model = CombinedModel(feature_extractor, face_detect_branch, person_detect_branch, face_rec_branch, pose_detect_branch)
torch.save(combined_model, os.path.join(filepath, "..", "edited_components", "combined_model.pth"))